In [1]:
import json
from selenium import webdriver
from bs4 import BeautifulSoup

import functions as fn

In [2]:
def get_html(driver, page):
    driver.get(page)
    html = driver.page_source
    return html

def get_ids(soup, id_list):
    """parse id of employee"""
    temp_id = []
    for a in soup.find_all('a', href=True):
        if "id" in a['href'] and "action=read" in a['href']:
            id = a['href'].split("=")[-1]
            if id not in id_list:
                temp_id.append(id)
    temp_id = list(set(temp_id))
    return temp_id

In [3]:

class City:
    def __init__(self, region_link, num_of_pages):
        self.link = "https://employmentcenter.ru/resume/?action=findit&region=" + region_link
        self.num_of_pages = num_of_pages
        self.id_list = []
    
    def turn_page(self, page_ind):
        page_link = self.link + "&nbl=" + str(page_ind)
        return page_link
    
class Person:
    def __init__(self, driver, id):
        self.id = id
        self.person_link = "https://employmentcenter.ru/resume/?action=read&id=" + id
        self.person_html = get_html(driver, self.person_link)
        self.bs = BeautifulSoup(self.person_html, "html.parser")

    @property
    def parse_data(self):
        item_list = ["address", "name", "gender", "Дата рождения", "Возраст", "telephone", "email", "jobTitle","Минимальный желательный оклад (зарплата)", "Желаемый график работы", "Образование"]
        parsed_list = []
        try:
            for item in item_list:
                if item == "telephone":
                    parsed_list.append(fn.obtain_phone(str(self.bs.find("span", itemprop=item))))
                elif item == "gender" or item =="jobTitle":
                    parsed_list.append(fn.obtain_name(str(self.bs.find("span", itemprop=item)))[1:-1])
                elif item == "Возраст" or item == "Образование" or item =='Желаемый график работы' or item == "Дата рождения":
                    parsed_list.append(fn.obtain_age_education(self.bs, item))
                elif item == "Минимальный желательный оклад (зарплата)":
                    parsed_list.append(fn.obtain_age_education(self.bs, item).split("/")[1][:-4])
                elif item == "name":
                    parsed_list.append(fn.edit_name(fn.obtain_name(str(self.bs.find("span", itemprop=item)))))                 
                else:
                    parsed_list.append(fn.obtain_name(str(self.bs.find("span", itemprop=item))))
            return fn.employee_dict(parsed_list)
        except:
            return None

In [4]:
# Список закодированных номеров каждого города: 
# Казань, Набережные Челны, Нижнекамск, Альметьевск, Зеленодольск, Чистополь, Иннополис, Бугульма, Лениногорск
cities_list = ["496", "834", "869", "37", "442", "1434", "5327", "188", "705"]
num_of_pages = 25

f = open("data.json", "r+", encoding='utf-8')
data = []
driver = webdriver.Edge()
for i in range(len(cities_list)):
    city = City(cities_list[i], num_of_pages)
    for j in range(num_of_pages):
        page_id_list = []
        
        try:
            city_soup = BeautifulSoup(get_html(driver, city.turn_page(j)), "html.parser")
            page_id_list = get_ids(city_soup, city.id_list)
            city.id_list += page_id_list
        except:
            break
        
        for id in page_id_list:
            person = Person(driver, id)
            f.seek(0)
            if person.parse_data != None:
                data.append(person.parse_data)
                json.dump(data, f, ensure_ascii=False, indent=4)                      
driver.close()